In [ ]:
Used sources are https://towardsdatascience.com/sentiment-analysis-with-deep-learning-62d4d0166ef6 and https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/.

In [ ]:

import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import pandas as pd
import nltk

nltk.download('punkt')

# fix random seed for reproducibility
np.random.seed(7)


# load cleaned data
reviews = pd.read_csv('../Data/simpler_clean_data.csv')


In [ ]:
y = reviews.iloc[:, :-1] 
X = reviews.iloc[:, -1] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state = 5)

In [ ]:
words = []

def word_cnt(text):
    tokens = nltk.word_tokenize(text)
    for word in tokens:
        if word not in words:
            words.append(word)

for text in X_train:
    word_cnt(text)
    

In [ ]:
new_x_train = []
new_x_test = []

def text_to_intlist(text, new):
    tokens = nltk.word_tokenize(str(text))
    list = []
    for word in tokens:
        list.append(words.index(word))
    new.append(list)
   
def reviews_to_ints(df, new):
    for text in X_train:
        text_to_intlist(text, new)

reviews_to_ints(X_train, new_x_train)
reviews_to_ints(X_test, new_x_test)

In [ ]:
X_train = np.array(new_x_train)
X_test = np.array(new_x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
# truncate and pad input sequences
max_review_length = 500
top_words = len(words)
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)

In [ ]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))